In [35]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

import time
from os.path  import basename
import requests
from PIL import Image
from IPython.display import display
import pandas as pd

In [36]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [37]:
features_df = pd.DataFrame(columns=['cena', 'zametr', 'pokoje', 'powierzchnia', 'umeblowane', 'czynsz', 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci', \
                                    'stanwykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie', 'opis'])

In [38]:
def features_to_df(input_string):
    
    possible_feaures = ['typ', 'zametr', 'powierzchnia', 'cena', 'piętro', 'umeblowane', 'rynek', 'rodzaj zabudowy', 'pokoje', 'czynsz', 'forma wlasnosci', \
                        'stanwykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie', 'opis']


    pairs = [pair.strip().split(':') for pair in input_string.rstrip(',').split(', ')]
    data_dict = {key.strip(): value.strip() for key, value in pairs}
    
    if 'Cena za m²' in data_dict.keys():
        data_dict['zametr'] = data_dict.pop('Cena za m²')
    if 'Poziom' in data_dict.keys():
        data_dict['Piętro'] = data_dict.pop('Poziom')
    if 'Liczba pokoi' in data_dict.keys():
        data_dict['pokoje'] = data_dict.pop('Liczba pokoi')

    data_dict_lower = {key.lower(): value for key, value in data_dict.items()}

    for feature in possible_feaures:
        if feature not in data_dict_lower.keys():
            data_dict_lower[feature] = None

    features_df.loc[len(features_df)] = data_dict_lower

In [39]:
input_string = 'Typ: Prywatne, Cena za m²: 11111.11 zł/m², Poziom: 3, Umeblowane: Tak, Rynek: Wtórny, Rodzaj zabudowy: Kamienica, Powierzchnia: 40,50 m², Liczba pokoi: 2 pokoje,'
str2 = 'typ: Prywatne, Cena za m²: 10151.52 zł/m², Poziom: 1, Umeblowane: Tak, Rynek: Wtórny, Rodzaj zabudowy: Blok, Powierzchnia: 66 m², Liczba pokoi: 2 pokoje,'

features_to_df(input_string)
features_to_df(str2)
features_df

,cena,zametr,pokoje,powierzchnia,umeblowane,czynsz,rodzaj zabudowy,rynek,piętro,typ,forma wlasnosci,stanwykonczenia,balkon/ogrod/taras,miejsce parkingowe,ogrzewanie,opis
0,None,11111.11 zł/m²,2 pokoje,"40,50 m²",Tak,None,Kamienica,Wtórny,3,Prywatne,None,None,None,None,None,None
1,None,10151.52 zł/m²,2 pokoje,66 m²,Tak,None,Blok,Wtórny,1,Prywatne,None,None,None,None,None,None


***OLX***

In [40]:
def get_info_olx(url):
    
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    driver.implicitly_wait(1)
    button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    button.click()

    cena = driver.find_element(By.CLASS_NAME, "css-93ez2t").text
    opis = driver.find_element(By.CLASS_NAME, "css-1m8mzwg").text.split("OPIS")[1].strip()

    tagi = driver.find_elements(By.CLASS_NAME, "css-1r0si1e")
    
    tags = 'typ: '
    for tag in tagi:
        tags = tags + tag.text + ', '

    tags = tags + 'cena: ' + cena
    #tags = tags + 'opis: ' + opis
    
    features_to_df(tags.strip())

    return features_df
    

In [41]:
get_info_olx('https://www.olx.pl/d/oferta/wynajme-mieszkanie-3-pokoje-57m2-wroclaw-CID3-IDYctw0.html')

,cena,zametr,pokoje,powierzchnia,umeblowane,czynsz,rodzaj zabudowy,rynek,piętro,typ,forma wlasnosci,stanwykonczenia,balkon/ogrod/taras,miejsce parkingowe,ogrzewanie,opis
0,None,11111.11 zł/m²,2 pokoje,"40,50 m²",Tak,None,Kamienica,Wtórny,3,Prywatne,None,None,None,None,None,None
1,None,10151.52 zł/m²,2 pokoje,66 m²,Tak,None,Blok,Wtórny,1,Prywatne,None,None,None,None,None,None
2,3 300 zł,None,3 pokoje,57 m²,Tak,None,Blok,None,1,Prywatne,None,None,None,None,None,None


***Otodom***

In [63]:
def get_info_otodom(url):
    
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    driver.implicitly_wait(1)
    button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    button.click()
    driver.maximize_window()
    
    nazwa = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[2]/header/h1').text
    cena = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[2]/header/strong').text
    
    try:
        cena = driver.find_element(By.XPATH, '').text
    except:
        cena = None
    
    try:
        zametr = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/header/div[3]').text
    except:
        zametr = None

    try:
        pokoje = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[3]/div[2]/div/a').text
    except:
        pokoje = None
    
    # try:
    #     lokalizacja = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/header/div[2]/a').text
    # except:
    #     lokalizacja = None
    
    try:
        powierzchnia = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[1]/div[2]/div').text
    except:
        powierzchnia = None
    
    try:
        umeblowane = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[10]/div[2]/div').text   # na otodom podpisane jako wyposażenie
    except:
        umeblowane = None
        
    try:
        czynsz = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[7]/div[2]/div').text
    except:
        czynsz = None
        
    try:
        rodzaj_zabudowy = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[5]/div[2]/div').text
    except:
        rodzaj_zabudowy  = None
    
    try:
        rynek = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[5]/div/div[1]/div[2]/div').text
    except:
        rynek  = None
    
    try:
        pietro = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[5]/div[2]/div').text
    except:
        pietro  = None

    try:
        typ = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[2]/div[2]/div').text   # typ oferenta
    except:
        typ = None
    
    try:
        forma_wlasnosci = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[2]/div[2]/div').text 
    except:
        forma_wlasnosci = None

    try:
        stan_wykonczenia = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[4]/div[2]/div').text 
    except:
        stan_wykonczenia = None

    try:
        balkon_ogrod_taras = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[6]/div[2]/div').text 
    except:
        balkon_ogrod_taras = None

    try:
        miejsce_parkingowe = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[8]/div[2]/span/span/div').text 
    except:
        miejsce_parkingowe = None

    try:
        ogrzewanie = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[10]/div[2]/div').text 
    except:
        ogrzewanie = None

    try:
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[2]/div/button')
        button.click()
    except:
        pass

    try:
        opis = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[2]/div/div/div').text
    except:
        opis = None
    if opis == 'Obsługa zdalna':
        opis = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[3]/div/div/div').text
        
    # col = ['nazwa', 'cena', 'zametr', 'lokalizacja', 'powierzchnia', 'pokoje', 'pietro', 'czynsz', 'forma wlasnosci', 'stanwykonczenia', \
    #                                 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie', 'opis']
    
    # ['cena', 'zametr', 'pokoje', 'powierzchnia', 'umeblowane', 'czynsz', 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci', \
    #                                 'stanwykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie', 'opis'])
    
    # dane = [[nazwa, cena, zametr, lokalizacja, powierzchnia, pokoje, pietro, czynsz, forma_wlasnosci, stan_wykonczenia, balkon_ogrod_taras, miesce_parkingowe, ogrzewanie, opis]]

    # otodom = pd.DataFrame([[cena, zametr, pokoje, powierzchnia, umeblowane, czynsz, rodzaj_zabudowy, rynek, pietro, typ, \
    #                         forma_wlasnosci, stan_wykonczenia, balkon_ogrod_taras, miejsce_parkingowe, ogrzewanie, opis]], 
    #                      columns = ['cena', 'zametr', 'pokoje', 'powierzchnia', 'umeblowane', 'czynsz', 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci', \
    #                        'stanwykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie', 'opis'])

    features_df.loc[len(features_df.index)] = [cena, zametr, pokoje, powierzchnia, umeblowane, czynsz, rodzaj_zabudowy, rynek, pietro, typ, \
                            forma_wlasnosci, stan_wykonczenia, balkon_ogrod_taras, miejsce_parkingowe, ogrzewanie, opis]

    return features_df

In [64]:
get_info_otodom('https://www.otodom.pl/pl/oferta/2m-mozliwosc-3m-centrum-swietna-lokalizacja-ID4ot4k.html')

,cena,zametr,pokoje,powierzchnia,umeblowane,czynsz,rodzaj zabudowy,rynek,piętro,typ,forma wlasnosci,stanwykonczenia,balkon/ogrod/taras,miejsce parkingowe,ogrzewanie,opis
0,None,11111.11 zł/m²,2 pokoje,"40,50 m²",Tak,None,Kamienica,Wtórny,3,Prywatne,None,None,None,None,None,None
1,None,10151.52 zł/m²,2 pokoje,66 m²,Tak,None,Blok,Wtórny,1,Prywatne,None,None,None,None,None,None
2,3 300 zł,None,3 pokoje,57 m²,Tak,None,Blok,None,1,Prywatne,None,None,None,None,None,None
3,None,9 463 zł/m²,2,"52,73 m²",None,450 zł,None,wtórny,parter/4,None,pełna własność,do remontu,None,None,miejskie,Przedstawiam Państwu 2-pokojowe mieszkanie w Ś...


***Funkcja do wyciagania linkow z olx***

In [ ]:
def get_links(url, strony):
    
    links = []
    
    for i in range (1,strony+1):
        urls = url  + '?page=' + str(i)
        driver = webdriver.Chrome(PATH)
        driver.get(url)
        driver.maximize_window()
        button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        button.click()
        linki = driver.find_elements(By.CLASS_NAME, "css-rc5s2u")
        
        for link in linki:
            links.append(link.get_attribute("href"))
            
    return links

In [ ]:
def get_info(linki):
    
    wynikiolx = pd.DataFrame()
    wynikioto = pd.DataFrame()
    odrzucone = []
    
    for link in linki:
        if 'otodom' in link:
            try:
                wynikioto = pd.concat([wynikioto, get_info_otodom(link)], ignore_index = True)
            except:
                odrzucone.append(link)
        if 'olx' in link:
            try:
                wynikiolx = pd.concat([wynikiolx, get_info_olx(link)], ignore_index = True)
            except:
                odrzucone.append(link)
    
    wynikiolx.to_csv('D:/Skrypty/IX semestr/MM w Pythonie/Projekt/olx.csv', sep = ',', index = False, encoding = 'utf-8') 
    wynikioto.to_csv('D:/Skrypty/IX semestr/MM w Pythonie/Projekt/otodom.csv', sep = ',', index = False, encoding = 'utf-8') 
    
    return odrzucone